In [ ]:
import sys
import os
import pandas as pd


evals_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
sys.path.insert(0, evals_dir)

from eval_utils import *  # Import all functions from eval_utils
from src.evaluator import Evaluator

evaluator = Evaluator()